<a href="https://colab.research.google.com/github/pratamastan/bangkit-capstone0143/blob/main/NLP_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
data1=pd.read_csv("/content/drive/MyDrive/Quora question pair dataset/train complete indo.csv")

In [ ]:
data1=data1.dropna()

In [ ]:
data1.isnull().values.any()

False

In [ ]:
data1.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...,0
1,1,3,4,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...,0
2,2,5,6,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...,0
3,3,7,8,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...,0
4,4,9,10,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?,0


In [ ]:
data1.question1.str.len().max()

717

In [ ]:
data1.question2.str.len().max()

1395

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1[["question1", "question2"]], data1["is_duplicate"], test_size=0.2, random_state=42)

In [ ]:
xt=X_train.to_numpy()
yt=y_train.to_numpy()

xte=X_test.to_numpy()
yte=y_test.to_numpy()

Q1_train = X_train["question1"].to_numpy()
Q2_train = X_train["question2"].to_numpy()
Q1_test = X_test["question1"].to_numpy()
Q2_test = X_test["question2"].to_numpy()

In [ ]:
Q1_train

array(['Apa sajakah cerita pendeknya?',
       'Apa jawaban terbaik untuk pertanyaan HR, "Anda berasal dari latar belakang ECE, jadi mengapa Anda ingin bergabung dengan perusahaan IT"?',
       'Bagaimana cara membulatkan 1,44 ke persen terdekat?', ...,
       'Siapa yang saat ini memenangkan pemilihan presiden?',
       'Apa sajakah film Telugu / Tamil / Hindi yang pemeran utamanya menjadi mafia atau tidak?',
       'Berapa peringkat yang dipotong untuk perguruan tinggi Kedokteran Rohtak dalam kuota negara bagian?'],
      dtype=object)

In [ ]:
data1["question1"][0]

'Apa panduan langkah demi langkah untuk berinvestasi di pasar saham di India?'

In [ ]:
data1[["question1","question2"]]

,question1,question2
0,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...
1,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...
2,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...
3,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...
4,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?
...,...,...
404285,Ada berapa kata kunci dalam bahasa pemrograman...,Ada berapa kata kunci dalam Bahasa Pemrograman...
404286,Apakah Anda percaya ada kehidupan setelah kema...,Benarkah ada kehidupan setelah kematian?
404287,Apa satu koin?,Koin apa ini?
404288,Berapa perkiraan biaya hidup tahunan selama be...,Saya mengalami sedikit masalah rambut rontok t...


In [ ]:
#Embedding for Indonesian language (vector with 128 dimension)
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string)
embed.trainable=False
#Model runtime will increase if tranable=True to about 900-1000s per epoch

In [ ]:
xt1=embed(Q1_train)
xt2=embed(Q2_train)
xte1=embed(Q1_test)
xte2=embed(Q2_test)

In [ ]:
xt1

<tf.Tensor: shape=(323430, 128), dtype=float32, numpy=
array([[ 0.03792422, -0.08203722, -0.03952134, ..., -0.03751129,
        -0.13026127, -0.21294448],
       [ 0.16049062,  0.05221158, -0.03756306, ..., -0.06843793,
        -0.16083518,  0.00526717],
       [ 0.00541904,  0.03682816, -0.10739181, ..., -0.05703003,
        -0.12414219, -0.05216019],
       ...,
       [ 0.17406467,  0.06946962, -0.02178098, ..., -0.05727712,
        -0.17851247, -0.00691927],
       [ 0.14288372, -0.16400667,  0.03347619, ...,  0.04388543,
        -0.12863201, -0.02391369],
       [-0.02106932, -0.22670205, -0.11702876, ..., -0.13742013,
        -0.11424292,  0.08969257]], dtype=float32)>

In [ ]:
##Will run 1 epoch about 2 minutes
model=tf.keras.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.BatchNormalization(),
                           tf.keras.layers.Dense(1, activation="sigmoid")
])
#model.build((1,2,128))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 2, 128)            98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 200)               13000     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(np.stack((xt1, xt2), axis=1),
                    yt,
                    epochs=5,
                    validation_data=(np.stack((xte1, xte2), axis=1), yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 127s - loss: 0.5713 - accuracy: 0.7028 - val_loss: 0.5203 - val_accuracy: 0.7332
Epoch 2/5
10108/10108 - 89s - loss: 0.4990 - accuracy: 0.7478 - val_loss: 0.4760 - val_accuracy: 0.7597
Epoch 3/5
10108/10108 - 87s - loss: 0.4668 - accuracy: 0.7676 - val_loss: 0.4600 - val_accuracy: 0.7681
Epoch 4/5
10108/10108 - 86s - loss: 0.4468 - accuracy: 0.7809 - val_loss: 0.4450 - val_accuracy: 0.7805
Epoch 5/5
10108/10108 - 86s - loss: 0.4299 - accuracy: 0.7923 - val_loss: 0.4362 - val_accuracy: 0.7875


In [ ]:
xt[0].shape

(2,)

In [ ]:
## 1 epoch is about 150-200s, added GRU Layer between the stacked Bidirectional LSTM layer
model3=tf.keras.Sequential([tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                            tf.keras.layers.GRU(64, return_sequences=True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(256,activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.BatchNormalization(),
                            tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model3.summary()

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history3 = model3.fit(np.stack((xt1, xt2), axis=1),
                    yt,
                    epochs=5,
                    validation_data=(np.stack((xte1, xte2), axis=1), yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 172s - loss: 0.5752 - accuracy: 0.7000 - val_loss: 0.5065 - val_accuracy: 0.7385
Epoch 2/5
10108/10108 - 166s - loss: 0.4993 - accuracy: 0.7466 - val_loss: 0.4812 - val_accuracy: 0.7582
Epoch 3/5
10108/10108 - 169s - loss: 0.4704 - accuracy: 0.7663 - val_loss: 0.4679 - val_accuracy: 0.7670
Epoch 4/5
10108/10108 - 171s - loss: 0.4508 - accuracy: 0.7796 - val_loss: 0.4520 - val_accuracy: 0.7793
Epoch 5/5
10108/10108 - 166s - loss: 0.4343 - accuracy: 0.7901 - val_loss: 0.4396 - val_accuracy: 0.7847


In [ ]:
## Model without Bidirectional LSTM model accuracy is only 66-72%, take 2 inputs, will run 1 epoch about 90s 
# ques1=tf.keras.Input(shape=(), dtype=tf.string)
# ques2=tf.keras.Input(shape=(), dtype=tf.string)

# qu1=embed(ques1)
# qu2=embed(ques2)
# attention=tf.keras.layers.Attention()([qu1,qu2])
# #attention=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(attention)
# #attention=tf.keras.layers.Attention()(attention)
# attention=tf.keras.layers.Flatten()(attention)
# attention=tf.keras.layers.Dense(300, activation="relu")(attention)
# merged = tf.keras.layers.Dense(200,activation='relu')(attention)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)
# merged = tf.keras.layers.Dense(200,activation='relu')(merged)
# merged = tf.keras.layers.Dropout(0.2)(merged)
# merged = tf.keras.layers.BatchNormalization()(merged)

# is_duplicate = tf.keras.layers.Dense(1, activation="sigmoid")(merged)
# model2 = tf.keras.models.Model(inputs=[ques1,ques2], outputs=is_duplicate)
# model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
attention (Attention)           (None, 128)          0           keras_layer[2][0]            

In [ ]:
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# history2 = model2.fit([Q1_train, Q2_train],
#                     yt,
#                     epochs=5,
#                     validation_data=([Q1_test,Q2_test], yte),
#                     verbose=2)

Epoch 1/5
10108/10108 - 87s - loss: 0.6506 - accuracy: 0.6347 - val_loss: 1.0374 - val_accuracy: 0.5443
Epoch 2/5
10108/10108 - 87s - loss: 0.6365 - accuracy: 0.6497 - val_loss: 0.6807 - val_accuracy: 0.6189
Epoch 3/5
10108/10108 - 89s - loss: 0.6330 - accuracy: 0.6540 - val_loss: 0.7115 - val_accuracy: 0.6095
Epoch 4/5
10108/10108 - 86s - loss: 0.6312 - accuracy: 0.6559 - val_loss: 0.8011 - val_accuracy: 0.5844
Epoch 5/5
10108/10108 - 89s - loss: 0.6305 - accuracy: 0.6563 - val_loss: 0.6913 - val_accuracy: 0.6308


In [ ]:
#Model with Bidirectional LSTM, Attention layer, take 2 inputs, will run about 150s for 1 epoch
ques1=tf.keras.Input(shape=(), dtype=tf.string)
ques2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(ques1)
qu2=embed(ques2)

q1=tf.expand_dims(qu1, axis=1)
q2=tf.expand_dims(qu2, axis=1)
q1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(q1)
q2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(q2)

attention=tf.keras.layers.Attention()([q1,q2])
#attention=tf.keras.layers.Flatten()(attention)
attention=tf.keras.layers.Dense(300, activation="relu")(attention)
#attention=tf.keras.layers.Reshape((2,128))(attention)

#merged = tf.keras.layers.Add([q1,attention])
#merged = tf.keras.layers.Flatten()(attention)
merged = tf.keras.layers.Dense(200,activation='relu')(attention)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)
merged = tf.keras.layers.Dense(200,activation='relu')(merged)
merged = tf.keras.layers.Dropout(0.2)(merged)
merged = tf.keras.layers.BatchNormalization()(merged)

is_duplicate = tf.keras.layers.Dense(1, activation="sigmoid")(merged)
model1 = tf.keras.models.Model(inputs=[ques1,ques2], outputs=is_duplicate)
model1.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_21[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
tf.expand_dims_6 (TFOpLambda)   (None, 1, 128)       0           keras_layer[10][0]         

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history1 = model1.fit([Q1_train, Q2_train],
                    yt,
                    epochs=5,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/5
10108/10108 - 158s - loss: 0.6065 - accuracy: 0.6718 - val_loss: 0.5756 - val_accuracy: 0.6970
Epoch 2/5
10108/10108 - 150s - loss: 0.5720 - accuracy: 0.7036 - val_loss: 0.5628 - val_accuracy: 0.7107
Epoch 3/5
10108/10108 - 150s - loss: 0.5572 - accuracy: 0.7153 - val_loss: 0.5507 - val_accuracy: 0.7214
Epoch 4/5
10108/10108 - 152s - loss: 0.5445 - accuracy: 0.7252 - val_loss: 0.5390 - val_accuracy: 0.7288
Epoch 5/5
10108/10108 - 149s - loss: 0.5343 - accuracy: 0.7333 - val_loss: 0.5355 - val_accuracy: 0.7312


In [ ]:
a=embed(["saya memukul kursi"])
a2=embed(["kursi dipukul oleh saya"])

In [ ]:
prediction1 = model1.predict([np.asarray(["dia memainkan gitar"]),np.asarray(["gitar dimainkan oleh dia"])])
print(prediction1)

[[[0.3716218]]]


In [ ]:
prediction = model.predict(np.stack((a,a2),axis=1))
print(prediction)

[[0.6698727]]


In [ ]:
prediction = model3.predict(np.stack((a,a2),axis=1))
print(prediction)

[[0.6323676]]


In [ ]:
# Model with Bidirectional LSTM, GRU, without Attention, will run for about 150-200s per epoch
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
qstn2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(qstn1)
qu2=embed(qstn2)
concat=tf.keras.layers.concatenate([qu1,qu2])
concat=tf.keras.layers.Reshape((2,128))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(concat)
concat=tf.keras.layers.GRU(64, return_sequences=True)(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
is_dupe=tf.keras.layers.Dense(1, activation="sigmoid")(concat)

model4 = tf.keras.models.Model(inputs=[qstn1,qstn2], outputs=is_dupe)
model4.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
concatenate_31 (Concatenate)    (None, 256)          0           keras_layer[14][0]         

In [ ]:
# a=embed(["abc"])
# b=embed(["bcd"])
# aa=tf.keras.layers.Reshape((1,128))(a)
# bb=tf.keras.layers.Reshape((1,128))(b)
# ab=tf.keras.layers.concatenate([a,b])
# tf.keras.layers.Reshape((2,128))(ab)

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history4 = model4.fit([Q1_train, Q2_train],
                    yt,
                    epochs=10,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/10
10108/10108 - 150s - loss: 0.4198 - accuracy: 0.7990 - val_loss: 0.4313 - val_accuracy: 0.7890
Epoch 2/10
10108/10108 - 152s - loss: 0.4079 - accuracy: 0.8069 - val_loss: 0.4330 - val_accuracy: 0.7894
Epoch 3/10
10108/10108 - 152s - loss: 0.3980 - accuracy: 0.8131 - val_loss: 0.4282 - val_accuracy: 0.7911
Epoch 4/10
10108/10108 - 151s - loss: 0.3872 - accuracy: 0.8188 - val_loss: 0.4216 - val_accuracy: 0.7967
Epoch 5/10
10108/10108 - 151s - loss: 0.3782 - accuracy: 0.8236 - val_loss: 0.4262 - val_accuracy: 0.7959
Epoch 6/10
10108/10108 - 151s - loss: 0.3688 - accuracy: 0.8293 - val_loss: 0.4221 - val_accuracy: 0.8013
Epoch 7/10
10108/10108 - 150s - loss: 0.3597 - accuracy: 0.8350 - val_loss: 0.4660 - val_accuracy: 0.7975
Epoch 8/10
10108/10108 - 149s - loss: 0.3524 - accuracy: 0.8396 - val_loss: 0.4424 - val_accuracy: 0.7912
Epoch 9/10
10108/10108 - 150s - loss: 0.3442 - accuracy: 0.8434 - val_loss: 0.4298 - val_accuracy: 0.8007
Epoch 10/10
10108/10108 - 154s - loss: 0.3364 

In [ ]:
prediction4 = model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
print(prediction4)

[[0.8118279]]
